<a href="https://colab.research.google.com/github/audio-bible/scripts/blob/master/Youtube_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Youtube Downloader

Download audio from [the KJV Audio Bible Channel on youtube](https://www.youtube.com/@skiesaboveus/playlists) of all bible chapters and save to google drive (My Drive/kjv_audio) sorted by chapters. Futher processing may be needed to make use of the audio.

You will likely exceed  google drive quota. So we suggest that once the download is done, you download the directory and do futher processing elsewhere



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install --upgrade audio-bible-tools


In [ ]:
from audio_bible_tools import YouTubeDownloader
root_dir = '/content/drive/My Drive/'
project_dir = 'kjv_audio'
channel_handle = 'skiesaboveus'
genesis_playlist = "01 Genesis Chapters 01-50 (KJV)"
revelation_playlist = "66 Revelation Chapters 01-22 (KJV)"

# Initialize the downloader
downloader = YouTubeDownloader(
    root_dir=root_dir,
    channel_handle=channel_handle,
    project_dir=project_dir,
    genesis_playlist=genesis_playlist,
    revelation_playlist=revelation_playlist
)

# Remove any temporary files
downloader.remove_tmp_files()

# Print the total number of files in the project directory
# print(downloader.count_files_in_directory())

# downloader.list_and_process_playlists()

# Remove temporary files again after download
downloader.remove_tmp_files()

# Process the downloaded files
downloader.version("kjv") \
    .voice("kjv-audio-bible") \
    .result_dir("bible_audio_processed") \
    .chapterPattern("Chapter (\d+)") \
    .process()

In [ ]:
from audio_bible_tools import YouTubeDownloader

root_dir = '/content/drive/My Drive/'
project_dir = 'kjv_audio_1'
channel_handle = 'ScourbyYouBible'
genesis_playlist = "1 ~ Book of Genesis by the Chapter"
revelation_playlist = "66 ~ Book of Revelation by the Chapter"

# Initialize the downloader
downloader = YouTubeDownloader(
    root_dir=root_dir,
    channel_handle=channel_handle,
    project_dir=project_dir,
    genesis_playlist=genesis_playlist,
    revelation_playlist=revelation_playlist
)

# Remove any temporary files
downloader.remove_tmp_files()

# Print the total number of files in the project directory
# print(downloader.count_files_in_directory())

# downloader.list_and_process_playlists()

# # Remove temporary files again after download
# downloader.remove_tmp_files()
# # downloader.download_youtube_audio("https://www.youtube.com/watch?v=EsUIUb0DjPM", "/content/drive/My Drive/kjv_audio_1/30_~_Book_of_Amos_by_the_Chapter")

# # Process the downloaded files
downloader.version("kjv") \
    .voice("scourby") \
    .result_dir("bible_audio_processed") \
    .chapterPattern("(\d+)\s+of\s+\d+\s+Chapters") \
    .process()

## Manual Sync To Github

In [2]:
from google.colab import userdata

# Retrieve the secret
GITHUB_TOKEN = userdata.get('REPO_SECRET_AUDIO_BIBLE')

# Ensure the secret is available as an environment variable
import os
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN

In [ ]:
%%bash
# Base directory containing the repositories
BASE_DIR="/content/drive/My Drive/bible_audio_processed"

rm -rf /tmp/cloned*

# Check if GITHUB_TOKEN is set
if [ -z "$GITHUB_TOKEN" ]; then
    echo "Error: GITHUB_TOKEN environment variable is not set."
    exit 1
fi
count=0
# Loop through all subdirectories in the base directory
for repo_dir in "$BASE_DIR"/*/; do
    if [ -d "$repo_dir" ]; then
        repo=$(basename "$repo_dir")
        echo "Processing repository: $repo"

        if [ ! -d "$repo_dir/.git" ]; then
            echo "No .git directory found. Cloning repository..."

            # Clone the repository
            stdbuf -oL git clone "https://$GITHUB_TOKEN@github.com/audio-bible/$repo.git" -b master "/tmp/cloned_$repo"

            if [ $? -ne 0 ]; then
                echo "Failed to clone $repo. Skipping..."
                continue
            fi

            # # Sync files from the source directory to the cloned repository
            # cd "/tmp/cloned_$repo"
            # stdbuf -oL rsync -av "$BASE_DIR/$repo/" ./

            # # Add all changes
            # stdbuf -oL git add .

            # # Set commit author
            # git config --local user.name "GitHub Bot"
            # git config --local user.email "bot@example.com"

            # # Commit and push changes
            # stdbuf -oL git commit -m "Update audio files for $repo"
            # stdbuf -oL git push origin master

            # # Sync back the .git directory to the original location
            rsync -av "/tmp/cloned_$repo/" "$BASE_DIR/$repo/"
            rm -rf "/tmp/cloned_$repo"

        fi
        # else
            echo ".git directory exists. Pulling latest changes and updating repository..."

            cd "$BASE_DIR/$repo/"
            # check if .github/workflows.deploy does not exist
            FILE_URL="https://raw.githubusercontent.com/audio-bible/scripts/master/deploy.yaml"
            if [ ! -f ".github/workflows/deploy.yaml" ]; then
              echo "MISSING YAML............."
              mkdir -p ".github/workflows"
              curl -sSL "$FILE_URL" -o ".github/workflows/deploy.yaml"
            fi

            # Add all changes
            stdbuf -oL git add .

            # Set commit author
            git config --local user.name "GitHub Bot"
            git config --local user.email "bot@example.com"

            # Commit and push changes
            stdbuf -oL git commit -m "Update audio files for $repo"
            stdbuf -oL git pull "https://$GITHUB_TOKEN@github.com/audio-bible/$repo.git" master --rebase
            stdbuf -oL git push "https://$GITHUB_TOKEN@github.com/audio-bible/$repo.git" master
        # fi
        # # increment count
        # count=$((count+1))
        # echo "count: $count"
        # if [ $count -gt 2 ]; then
        #     break
        # fi
        echo "Completed processing $repo"
        echo "------------------------"

    fi
done


In [31]:
%%bash
ls -lha "/content/drive/My Drive/bible_audio_processed/1_Corinthians/.git"